In [14]:
import numpy as np
import matplotlib.pyplot as plt
import time
import imageio.v2 as imageio
from scipy.fftpack import fft, fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.linalg import solve, solve_triangular, lu
from scipy.sparse.linalg import bicgstab, gmres
from scipy.sparse import spdiags


In [15]:
#+++++++++++++++ 1. Parameters
tspan = np.arange(0, 50.5, 0.5)
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny


In [16]:
#+++++++++++++++  2. Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)


In [17]:
#+++++++++++++++  3. Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

In [18]:
#+++++++++++++++  4. Return Code from HW 4
# Parameters
m = 64
L = 20
dx = L / m
n = m * m

e0 = np.zeros(n)
e1 = np.ones(n)
e2 = np.copy(e1)
e4 = np.copy(e0)

for j in range(1, m+1):
    e2[m*j - 1] = 0 
    e4[m*j - 1] = 1

# Adjusted vectors for diagonals
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]

e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]

# Construct Matrix a (Laplacian)
diagonals_A = [e1, e1, e5, e2, -4 * e1, e3, e4, e1, e1]
offsets_A = [-(n - m), -m, -m + 1, -1, 0, 1, m - 1, m, (n - m)]
A = (spdiags(diagonals_A, offsets_A, n, n) / (dx**2)).toarray()

# Construct Matrix C (Partial derivative with respect to y)
diagonals_B = [e1, -e1, e1, -e1]
offsets_B = [-(n - m),-m, m, (n - m)]
B = (spdiags(diagonals_B, offsets_B, n, n) / (2 * dx)).toarray()

# Construct Matrix b (Partial derivative with respect to x)
diagonals_C = [e5, -e2, e3, -e4]
offsets_C = [-m + 1, -1, 1, m - 1]
C = (spdiags(diagonals_C, offsets_C, n, n) / (2 * dx)).toarray()

A[0, 0] = 2


In [ ]:
# Part C

w1 = (np.exp(-(X+4)**2 - Y**2 / 20)).flatten()
w2 = -1 * (np.exp(-(X-4)**2 - Y**2 / 20)).flatten()
w3 = -1 * w1
w4 = -1 * w2

# FFT Method

#+++++++++++++++  5. Define the PDE system
def spc_rhs(t, w, nx, ny, K, nu):
    wtc = w.reshape((nx, ny))
    wt = fft2(wtc)
    psit = -wt / K
    psi = np.real(ifft2(psit))
    psi = psi.flatten()
    rhs = nu * np.dot(A, w) - np.dot(B, psi) * np.dot(C, w) + np.dot(C, psi) * np.dot(B, w)
    return rhs

#+++++++++++++++ 6. Solution
start_time = time.time()
wtsol = solve_ivp(spc_rhs, (tspan[0], tspan[-1]), w1, t_eval=tspan, method = 'RK45', args = (nx, ny, K, nu))
wtsol2 = solve_ivp(spc_rhs, (tspan[0], tspan[-1]), w2, t_eval=tspan, method = 'RK45', args = (nx, ny, K, nu))
wtsol3 = solve_ivp(spc_rhs, (tspan[0], tspan[-1]), w3, t_eval=tspan, method = 'RK45', args = (nx, ny, K, nu))
wtsol4 = solve_ivp(spc_rhs, (tspan[0], tspan[-1]), w4, t_eval=tspan, method = 'RK45', args = (nx, ny, K, nu))
A1w1 = wtsol.y
A1w2 = wtsol2.y
A1w3 = wtsol3.y
A1w4 = wtsol3.y
end_time = time.time()

fft_time = end_time - start_time

# print(A1w1)
print("Time to run fft : ", fft_time)

# # # +++++++++++++++ Plot the solution at each time step
# for j, t in enumerate(tspan):
#     wtc = A1[:, j].reshape((ny, nx)) 
#     # wtc2 = A1w2[:, j].reshape((ny, nx)) 
#     plt.subplot(3, 3, j + 1)
#     plt.pcolor(x, y, wtc, shading='auto', cmap='gnuplot')
#     # plt.pcolor(x, y, wtc2, shading='auto', cmap='gnuplot')
#     plt.title(f'Time: {t}')
#     plt.colorbar()

# plt.tight_layout()
# plt.show()

Time to run fft :  16.094664812088013


In [ ]:
# #  +++++++++++++++  Two opposite “charged”
gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(tspan):
    wtc1 = A1w1[:, j].reshape((ny, nx)) 
    wtc2 = A1w2[:, j].reshape((ny, nx)) 
    wtc3 = A1w3[:, j].reshape((ny, nx)) 
    wtc4 = A1w4[:, j].reshape((ny, nx)) 
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.pcolor(x, y, wtc1+wtc2, shading='auto', cmap='inferno')
    fig.colorbar(c)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('opposite_charge.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames

# Optional: Remove the frame image after gif creation
import os
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!


In [ ]:
# #  +++++++++++++++  Same “charged” with (-) (-)
gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(tspan):
    wtc1 = A1w1[:, j].reshape((ny, nx)) 
    wtc2 = A1w2[:, j].reshape((ny, nx)) 
    wtc3 = A1w3[:, j].reshape((ny, nx)) 
    wtc4 = A1w4[:, j].reshape((ny, nx)) 
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.pcolor(x, y, wtc1+wtc4, shading='auto', cmap='inferno')
    fig.colorbar(c)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('same_negative_charge.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames

# Optional: Remove the frame image after gif creation
import os
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!


In [ ]:
# #  +++++++++++++++  Two pairs of oppositely “charged”
gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(tspan):
    wtc1 = A1w1[:, j].reshape((ny, nx)) 
    wtc2 = A1w2[:, j].reshape((ny, nx)) 
    wtc3 = A1w3[:, j].reshape((ny, nx)) 
    wtc4 = A1w4[:, j].reshape((ny, nx)) 
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.pcolor(x, y, wtc1+wtc4, shading='auto', cmap='inferno')
    fig.colorbar(c)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('same_positive_charge.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames

# Optional: Remove the frame image after gif creation
import os
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!
